In [ ]:
from selenium import webdriver
import pymongo, time
from random import randint


def facebook(username, password):
    proxie = {'https': 'https://127.0.0.1:1080'}
    prefs = {'profile.default_content_setting_values':
        {
            'notifications': 2
        }
    }
    options = webdriver.ChromeOptions()
    options.add_argument('--proxy-server=http://127.0.0.1:1080')
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get('https://www.facebook.com/')

    driver.find_element_by_id('email').send_keys(username)
    driver.find_element_by_id('pass').send_keys(password)
    driver.find_element_by_id('loginbutton').submit()
    driver.implicitly_wait(200)
    driver.get(url)
    for i in range(700):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        print('loop times:', i + 1)
        time.sleep(randint(2, 4))
        i += 1
    time.sleep(4)
    items = driver.find_elements_by_css_selector('#groupsMemberSection_recently_joined ._60ri')
    for item in items:
        nick_name = item.text
        profile_link = item.find_element_by_tag_name('a').get_attribute('href')
        print(nick_name, profile_link)
        data = {
            'nickname': nick_name,
            'profile_link': profile_link
        }
        save_to_mongo(data, nick_name)


def save_to_mongo(result, nick_name):
    MONGO_URL = 'localhost'
    MONGO_DB = 'FBcrawl'
    MONGO_COLLECTION = 'groupmember_0428'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]

    try:
        collection.update({'nickname': nick_name}, {'$set': result}, True)
        print('save to mongodb')
    except Exception as err:
        print('failed to save to mongodb')
        print(err)


if __name__ == '__main__':
    url = 'https://www.facebook.com/groups/491704807888001/members/'
    username = ''
    password = ''

    facebook(username, password)